[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%2010/Worksheet%2010-2%20Finite%20Difference%20Methods%20for%20BVPs.ipynb)

# Worksheet 10.2 Finite Difference Method for Boundary Value Problems
## ENGR 240: Engineering Computations

## Introduction: Beam Deflection Analysis

In structural engineering, understanding beam deflection is critical for ensuring safe and reliable designs. This worksheet explores the **finite difference method** for solving boundary value problems through the analysis of a simply supported beam with foundation settlement.

### Physical System

**Simply Supported Beam Specifications:**
- **Length**: 4.0 m
- **Material**: Steel (E = 200 × 10⁹ Pa)
- **Moment of Inertia**: I = 8.33 × 10⁻⁶ m⁴
- **Loading**: Uniform distributed load w = 1000 N/m (downward)
- **Support conditions**: Left pinned, right settled by δ = 5 mm

### Mathematical Model

The governing differential equation for beam deflection starts with the fourth-order equation:
$EI\frac{d^4y}{dx^4} = w(x)$

For our uniform distributed load case with $w(x) = w$, integrating twice:
- First integration: $EI\frac{d^3y}{dx^3} = wx + C_1$ (shear force relationship)
- Second integration: $EI\frac{d^2y}{dx^2} = \frac{wx^2}{2} + C_1x + C_2$ (bending moment $M(x)$)

For a simply supported beam, the bending moment must be zero at both ends:
- At $x = 0$: $M(0) = C_2 = 0$
- At $x = L$: $M(L) = \frac{wL^2}{2} + C_1L = 0 \Rightarrow C_1 = -\frac{wL}{2}$

Therefore, the bending moment equation becomes:
$EI\frac{d^2y}{dx^2} = \frac{wLx}{2} - \frac{wx^2}{2} = \frac{wx}{2}(L - x)$

**Boundary Conditions:**
- **Left support (x = 0)**: $y(0) = 0$ (pinned connection)
- **Right support (x = L)**: $y(L) = \delta = 0.005$ m (foundation settlement)

**Learning Objectives:**
- Apply finite difference approximations to boundary value problems
- Formulate coefficient matrices for linear systems by hand
- Understand how boundary conditions modify the linear system
- Investigate mesh refinement and convergence to analytical solutions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
print("Libraries imported successfully!")

## System Parameters

In [ ]:
# Physical parameters
L = 4.0           # Beam length (m)
w = 1000.0        # Distributed load (N/m)
E = 200e9         # Young's modulus (Pa)
I = 8.33e-6       # Moment of inertia (m^4)
delta = 0.005     # Settlement at right end (m)

# Display parameters
print("=== Beam Deflection Analysis Parameters ===")
print(f"Length: {L} m")
print(f"Distributed load: {w} N/m")
print(f"Young's modulus: {E/1e9:.0f} GPa")
print(f"Moment of inertia: {I*1e6:.2f} × 10⁻⁶ m⁴")
print(f"Settlement: {delta*1000:.0f} mm")
print(f"\nCorrect ODE: EI d²y/dx² = (wx/2)(L-x)")
print("This accounts for proper simply supported boundary conditions.")

## Task 1: Problem Setup and Analytical Solution (10 minutes)

Before applying the finite difference method, let's establish our reference solution. For this beam problem, the analytical solution accounts for the correct simply supported boundary conditions on bending moment.

In [ ]:
def analytical_solution(x):
    """
    Analytical solution for simply supported beam with uniform load and settlement.
    
    Based on the correct ODE: EI d²y/dx² = (wx/2)(L-x)
    Integrating twice and applying boundary conditions gives the classical
    simply supported beam deflection formula plus settlement effect.
    
    Components:
    - Classical beam deflection: -(w/24EI) * x * (L³ - 2L²x + x³)
    - Linear settlement variation: δ * x / L
    """
    # Classical simply supported beam deflection (accounts for moment BCs)
    beam_deflection = -(w/(24*E*I)) * x * (L**3 - 2*L**2*x + x**3)
    
    # Linear settlement variation from left (0) to right (δ)
    settlement_effect = delta * x / L
    
    return beam_deflection + settlement_effect

# Generate analytical solution for plotting
x_analytical = np.linspace(0, L, 200)
y_analytical = analytical_solution(x_analytical)

# Plot the reference solution
plt.figure(figsize=(12, 6))
plt.plot(x_analytical, y_analytical*1000, 'b-', linewidth=3, label='Analytical Solution')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axhline(y=delta*1000, color='r', linestyle=':', alpha=0.5, label=f'Settlement = {delta*1000:.0f} mm')

plt.xlabel('Position x (m)', fontsize=12)
plt.ylabel('Deflection y (mm)', fontsize=12)
plt.title('Beam Deflection: Analytical Reference Solution', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Key results
max_deflection = np.min(y_analytical)
max_location = x_analytical[np.argmin(y_analytical)]
print(f"\n=== Analytical Solution Results ===")
print(f"Maximum deflection: {max_deflection*1000:.2f} mm")
print(f"Location of maximum: {max_location:.2f} m from left support")
print(f"Deflection at right end: {delta*1000:.1f} mm (settlement)")
print(f"\nNote: This solution satisfies both displacement and moment boundary conditions.")

## Task 2: Finite Difference Fundamentals (8 minutes)

### The Challenge: Variable Coefficient ODE

Our governing equation is:
$EI\frac{d^2y}{dx^2} = \frac{wx}{2}(L-x)$

This has a **variable right-hand side** that depends on position $x$. At each grid point $x_i$, we need:
$\frac{d^2y}{dx^2}\bigg|_{x=x_i} = \frac{w x_i}{2EI}(L-x_i)$

### Finite Difference Approximation

Using the second-order central difference:
$\frac{d^2y}{dx^2}\bigg|_{x=x_i} \approx \frac{y_{i+1} - 2y_i + y_{i-1}}{h^2}$

### Grid Discretization

For our 6-node system:
- **Grid spacing**: $h = L/(n-1) = 4.0/5 = 0.8$ m  
- **Node locations**: x₀ = 0, x₁ = 0.8, x₂ = 1.6, x₃ = 2.4, x₄ = 3.2, x₅ = 4.0
- **Known values**: y₀ = 0 (left BC), y₅ = δ (right BC)
- **Unknowns**: y₁, y₂, y₃, y₄ (4 interior nodes → 4 equations needed)

In [ ]:
# 6-node grid setup
n_nodes = 6
h = L / (n_nodes - 1)
x_nodes = np.linspace(0, L, n_nodes)

print("=== 6-Node Finite Difference Grid ===")
print(f"Grid spacing: h = {h:.1f} m")
print(f"Number of interior nodes: {n_nodes-2}")
print("\nNode Information:")
print("Node    x (m)    Condition          RHS Value")
print("-" * 55)
for i, x in enumerate(x_nodes):
    if i == 0:
        condition = "y = 0 (pinned)"
        rhs_info = "N/A"
    elif i == n_nodes-1:
        condition = f"y = {delta*1000:.0f} mm (settlement)"
        rhs_info = "N/A"
    else:
        condition = "unknown (interior)"
        rhs_val = (w * x / (2 * E * I)) * (L - x) * h**2
        rhs_info = f"{rhs_val:.2e}"
    print(f"{i:4d}    {x:4.1f}     {condition:20s} {rhs_info}")

print(f"\n→ Need {n_nodes-2} equations for {n_nodes-2} unknowns")
print("→ Each interior node has a different RHS value due to variable loading")

## Task 3: Hand Formulation of Linear System (20 minutes)

### Step 1: Apply Finite Difference Approximation

At each interior node $i$, substitute the finite difference approximation:

$\frac{y_{i+1} - 2y_i + y_{i-1}}{h^2} = \frac{w x_i}{2EI}(L-x_i)$

Rearranging: $y_{i-1} - 2y_i + y_{i+1} = \frac{w x_i h^2}{2EI}(L-x_i)$

### Step 2: Write Equations for Interior Nodes

Apply this formula at nodes 1, 2, 3, and 4:

In [ ]:
# Calculate the RHS constants for each interior node
print("=== Finite Difference Equations (Variable RHS) ===")
print("\nInterior node equations (before applying boundary conditions):")

rhs_values = []
for i in range(1, n_nodes-1):  # Interior nodes 1, 2, 3, 4
    x_i = x_nodes[i]
    rhs_i = (w * x_i * h**2) / (2 * E * I) * (L - x_i)
    rhs_values.append(rhs_i)
    print(f"Node {i}: y_{i-1} - 2y_{i} + y_{i+1} = {rhs_i:.6f}")
    print(f"         (RHS from x={x_i:.1f}: wx(L-x)h²/2EI = {rhs_i:.2e})")

print("\nAfter applying boundary conditions (y₀ = 0, y₅ = δ):")
print(f"Node 1: 0 - 2y₁ + y₂ = {rhs_values[0]:.6f}  →  -2y₁ + y₂ = {rhs_values[0]:.6f}")
print(f"Node 2: y₁ - 2y₂ + y₃ = {rhs_values[1]:.6f}")
print(f"Node 3: y₂ - 2y₃ + y₄ = {rhs_values[2]:.6f}")
print(f"Node 4: y₃ - 2y₄ + δ = {rhs_values[3]:.6f}  →  y₃ - 2y₄ = {rhs_values[3] - delta:.6f}")

print("\nKey Insight: Variable loading creates different RHS values at each node!")

### Step 3: Matrix Formulation

**The coefficient matrix A remains the same** (tridiagonal structure), but **the RHS vector b has different values** at each interior node.

**Solution:**

$\mathbf{A} = \begin{bmatrix}
-2 & 1 & 0 & 0 \\
1 & -2 & 1 & 0 \\
0 & 1 & -2 & 1 \\
0 & 0 & 1 & -2
\end{bmatrix}, \quad
\mathbf{b} = \begin{bmatrix}
\frac{wx_1h^2}{2EI}(L-x_1) \\
\frac{wx_2h^2}{2EI}(L-x_2) \\
\frac{wx_3h^2}{2EI}(L-x_3) \\
\frac{wx_4h^2}{2EI}(L-x_4) - \delta
\end{bmatrix}$

In [ ]:
# Construct the coefficient matrix A (same tridiagonal structure)
A_hand = np.array([
    [-2.0,  1.0,  0.0,  0.0],   # Node 1 equation
    [ 1.0, -2.0,  1.0,  0.0],   # Node 2 equation  
    [ 0.0,  1.0, -2.0,  1.0],   # Node 3 equation
    [ 0.0,  0.0,  1.0, -2.0]    # Node 4 equation
])

# Construct the RHS vector b (now with variable values)
b_hand = np.array([
    rhs_values[0],              # Node 1: variable RHS
    rhs_values[1],              # Node 2: variable RHS
    rhs_values[2],              # Node 3: variable RHS
    rhs_values[3] - delta       # Node 4: variable RHS - δ (boundary effect!)
])

print("=== Hand-Calculated Linear System (Variable Loading) ===")
print("\nCoefficient matrix A (unchanged):")
print(A_hand)
print("\nRHS vector b (now variable):")
for i, val in enumerate(b_hand):
    node_num = i + 1
    if i == 3:
        print(f"[{val:8.6f}]  ← Node {node_num}: includes settlement effect")
    else:
        print(f"[{val:8.6f}]  ← Node {node_num}: wx(L-x)h²/2EI")

print(f"\nKey Difference: Each RHS value accounts for the local loading at that node position.")
print("The simply supported beam has spatially varying curvature, not constant curvature!")

## Task 4: Solution and Comparison (12 minutes)

In [ ]:
# Solve the linear system
y_interior = np.linalg.solve(A_hand, b_hand)

print("=== Linear System Solution ===")
print("Interior node deflections:")
for i, deflection in enumerate(y_interior, 1):
    print(f"y_{i} = {deflection:8.6f} m = {deflection*1000:7.3f} mm")

# Construct complete solution vector
y_fd_complete = np.zeros(n_nodes)
y_fd_complete[0] = 0.0           # Left BC: y₀ = 0
y_fd_complete[1:5] = y_interior  # Interior solutions
y_fd_complete[5] = delta         # Right BC: y₅ = δ

print(f"\nComplete nodal solution:")
print("Node    x (m)    FD Solution (mm)    Analytical (mm)    Error (mm)")
print("-" * 70)

y_analytical_nodes = analytical_solution(x_nodes)
for i, (x, y_fd, y_ana) in enumerate(zip(x_nodes, y_fd_complete, y_analytical_nodes)):
    error = (y_fd - y_ana) * 1000
    print(f"{i:4d}    {x:4.1f}     {y_fd*1000:10.3f}       {y_ana*1000:10.3f}      {error:+7.3f}")

max_error = np.max(np.abs(y_fd_complete - y_analytical_nodes)) * 1000
print(f"\nMaximum error: {max_error:.4f} mm")
print("The finite difference solution closely matches the analytical solution!")

## Engineering Insights and Summary

### Key Learning Points

**Mathematical Modeling:**
- Simply supported beams require zero bending moment boundary conditions
- This leads to the correct ODE: $EI\frac{d^2y}{dx^2} = \frac{wx}{2}(L-x)$
- The RHS varies with position, creating a **variable coefficient BVP**

**Finite Difference Method:**
- Coefficient matrix structure remains the same (tridiagonal)
- RHS vector values change at each interior node
- Each node accounts for local loading conditions
- Boundary conditions still modify the RHS vector

**Engineering Applications:**
- Foundation settlement analysis with proper beam theory
- Variable loading distributions
- Different support conditions and boundary constraints
- Validates importance of correct mathematical modeling

**Convergence Behavior:**
- Error still decreases as O(h²) for 2nd-order finite differences
- Accurate solutions even with coarse grids
- Proper physics leads to better numerical behavior